## Prepare dataset

In [1]:
import os
from PIL import Image

from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision import transforms, utils, models
import torch

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#img = Image.open('edges2shoes/train/10000_AB.jpg')
#plt.imshow(data_transform(img.crop((256, 0, 512, 256))).data.numpy().transpose(1, 2, 0))

In [4]:
# https://www.kaggle.com/pinocookie/pytorch-dataset-and-dataloader

class DatasetPix2Pix:
    def __init__(self, data_path):
        self.filenames = [os.path.join(data_path, filename) for filename in os.listdir(data_path)]
        
        self.data_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        image = Image.open(self.filenames[index])
        
        left = image.crop((0, 0, 256, 256))
        right = image.crop((256, 0, 512, 256))
        return self.data_transform(left),\
               self.data_transform(right)
    
    def __len__(self):
        return len(self.filenames)
    
    def __add__(self, other):
        return ConcatDataset([self, other])

In [6]:
data = DatasetPix2Pix('edges2shoes/train/')
dataloader = DataLoader(data, batch_size=8, shuffle=True, num_workers=4)

In [117]:
resnet = torchvision.models.resnet34(pretrained=True).cuda()

In [7]:
import torchvision

In [9]:
class Vectorizer(torch.nn.Module):
    def __init__(self, enc_size=256):
        super(self.__class__, self).__init__()
        
        self.resnet = torchvision.models.resnet34(pretrained=True)
        
        self.code = torch.nn.Sequential(torch.nn.Linear(1000, 512), 
                                        torch.nn.ReLU(), 
                                        torch.nn.Linear(512, enc_size))
        
    def forward(self, x):
        resnet_code = self.resnet(x)
        return self.code(resnet_code)

In [16]:
class Siamize(torch.nn.Module):
    def __init__(self, enc_size=256):
        super(self.__class__, self).__init__()
        
        self.vec1 = Vectorizer(enc_size)
        self.vec2 = Vectorizer(enc_size)
        
    def forward(self, x, y):
        return self.vec1(x), self.vec2(y)

In [17]:
siam = Siamize()

In [51]:
!git status

On branch master

Initial commit

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   .gitignore
	new file:   Untitled.ipynb



In [48]:
!cat .gitignore

ipynb_checkpoints/
edges2shoes
edges2shoes/
.ipynb_checkpoints/


In [47]:
!echo '.ipynb_checkpoints/' >> .gitignore